In [16]:
import numpy as np
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Khai báo trình duyệt Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Mở URL
driver.get("https://vnexpress.net/goc-nhin")
driver.maximize_window()

# Hàm cuộn trang
def scroll_page():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(random.uniform(2, 4))

# Click vào href và sleep 2 giây
def click_and_wait(title_element):
    try:
        driver.execute_script("arguments[0].scrollIntoView(true);", title_element)  # Cuộn tới phần tử
        sleep(1)  # Chờ để phần tử hiển thị đầy đủ
        title_element.click()  # Click vào liên kết
        sleep(2)  # Chờ 2 giây
    except ElementNotInteractableException:
        print("Không thể click vào liên kết.")

def close_popups():
    try:
        popup = driver.find_element(By.CSS_SELECTOR, "selector_của_popup")  # Xác định phần tử popup
        popup.click()
        print("Đã đóng pop-up.")
    except NoSuchElementException:
        pass  # Không có pop-up nào

# Hàm để trích xuất thông tin thẻ HTML sau khi click
def extract_detail_info():
    try:
    
        # Lấy thông tin "date"
        date_element = driver.find_element(By.CSS_SELECTOR, "span.date")
        date = date_element.text

        # Lấy thông tin "h1.title-detail"
        title_detail_element = driver.find_element(By.CSS_SELECTOR, "h1.title-detail")
        detailed_title = title_detail_element.text

        # Lấy thông tin "p.description"
        description_element = driver.find_element(By.CSS_SELECTOR, "p.description")
        detailed_description = description_element.text

        # Lấy thông tin nội dung bài viết
        content_paragraphs = driver.find_elements(By.CSS_SELECTOR, "article.fck_detail p")
        content = "\n".join([paragraph.text for paragraph in content_paragraphs])

        return {
            "Date": date,
            "Detailed Title": detailed_title,
            "Author's Position": detailed_description,
            "Content": content
        }
    except NoSuchElementException:
        print("Không tìm thấy thông tin chi tiết.")
        return {}

# Cập nhật hàm extract_articles để chỉ lấy 3 bài viết
def extract_articles(limit=3):
    articles = driver.find_elements(By.CSS_SELECTOR, "article.item-news")
    data = []
    count = 0  # Đếm số bài viết đã xử lý
    for article in articles:
        close_popups()  # Đóng pop-up nếu có

        if count >= limit:
            break  # Dừng lại khi đã thu thập đủ số bài viết
        try:
            # Tiêu đề và link
            title_element = article.find_element(By.CSS_SELECTOR, "h3.title-news a")
            title = title_element.text
            link = title_element.get_attribute("href")

            # Click vào liên kết và chờ 2 giây
            click_and_wait(title_element)

            # Lấy thông tin chi tiết từ bài viết
            detail_info = extract_detail_info()

            # Trở lại trang chính
            driver.back()
            sleep(2)

            # Gộp thông tin
            data.append({
                "Title": title,
                "Link": link,
                **detail_info
            })
            count += 1  # Tăng bộ đếm sau khi xử lý xong một bài viết
        except NoSuchElementException:
            # Nếu không tìm thấy phần tử, bỏ qua
            continue
    return data

# Cuộn và lấy dữ liệu
all_data = []
for _ in range(1):  # Cuộn 5 lần
    scroll_page()
    articles_data = extract_articles(limit=3)
    all_data.extend(articles_data)

# Loại bỏ trùng lặp
df = pd.DataFrame(all_data).drop_duplicates()

ElementClickInterceptedException: Message: element click intercepted: Element <a data-medium="Item-1" data-thumb="0" href="https://vnexpress.net/visa-qua-canh-ngat-ngheo-4831587.html" title="...">Visa quá cảnh ngặt nghèo</a> is not clickable at point (580, 29). Other element would receive the click: <span onclick="location.href='/thoi-tiet/tp-hcm#vn_source=Folder-GocNhin&amp;vn_campaign=Header&amp;vn_medium=Item-ThoiTiet&amp;vn_term=Desktop'">...</span>
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x004BEC13+23731]
	(No symbol) [0x0044C394]
	(No symbol) [0x0032BE63]
	(No symbol) [0x00375C97]
	(No symbol) [0x003740E9]
	(No symbol) [0x00371D0D]
	(No symbol) [0x00371022]
	(No symbol) [0x00365F57]
	(No symbol) [0x00391E5C]
	(No symbol) [0x003659A4]
	(No symbol) [0x003920F4]
	(No symbol) [0x003AB46E]
	(No symbol) [0x00391BF6]
	(No symbol) [0x00363F35]
	(No symbol) [0x00364EBD]
	GetHandleVerifier [0x0079F0D3+3039603]
	GetHandleVerifier [0x007B2DEA+3120778]
	GetHand

In [14]:
# Lưu vào file Excel
df.to_excel("D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles.xlsx")
# Đóng trình duyệt
driver.quit()
